In [ ]:
# Importing required libraries

import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Input.xlsx to Input.xlsx


In [ ]:
URL_DF = pd.read_excel('Input.xlsx')

In [ ]:
URL_DF.head()

,URL_ID,URL
0,123.0,https://insights.blackcoffer.com/rise-of-telem...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...


In [ ]:
URL_DF.isnull().sum()

URL_ID    0
URL       0
dtype: int64

In [ ]:
# Web scraping of weblinks present in Input file to text file

for i,r in URL_DF.iterrows():
  url = r['URL']
  url_id = r['URL_ID']
  page = requests.get(url)

  try:
    soup = BeautifulSoup(page.content,'lxml')
  except:
    print('No content present for {}'.format(url_id))

  try:
    title = soup.find('h1').get_text()
  except:
    print('No title present for {}'.format(url_id))

  a = ''
  try:
    for i in soup.find('article'):
      a += i.get_text()
  except:
    print('No content present for {}'.format(url_id))

  filename = '/content/drive/MyDrive/Blackcoffer/Scrapped_files/' + str(url_id) + '.txt'
  with open(filename,'w') as file:
    file.write(title + '\n' + a)


No title present for 11668.0
No content present for 11668.0
No title present for 17671.4
No content present for 17671.4


In [ ]:
# Path for directories
scrap_text = '/content/drive/MyDrive/Blackcoffer/Scrapped_files/'
master_dict = '/content/drive/MyDrive/Blackcoffer/Master_Dict'
stop_Words = '/content/drive/MyDrive/Blackcoffer/Stop_Words'

# Stop words
stop = set()
for file in os.listdir(stop_Words):
  with open(os.path.join(stop_Words,file),'r',encoding='ISO-8859-1') as f:
    stop.update(set(f.read().splitlines()))


positive = set()
negative = set()

for file in os.listdir(master_dict):
  if file == 'positive-words.txt':
    with open(os.path.join(master_dict,file),'r',encoding='ISO-8859-1') as f:
      positive.update(set(f.read().splitlines()))


for file in os.listdir(master_dict):
  if file == 'negative-words.txt':
    with open(os.path.join(master_dict,file),'r',encoding='ISO-8859-1') as f:
      negative.update(set(f.read().splitlines()))

# Scrapped files

scrap_docs = []
for file in os.listdir(scrap_text):
  with open(os.path.join(scrap_text,file),'r',encoding='ISO-8859-1') as f:
    text = f.read()
    words = word_tokenize(text)
    finalized_text = [word for word in words if word.lower() not in stop]
    scrap_docs.append(finalized_text)

In [ ]:
# Variables calculations

pos_words = []
neg_words = []
positive_score = []
negative_score = []
polarity_score = []
subjectivity_score = []


for doc in range(len(scrap_docs)):
  pos_words.append([word for word in scrap_docs[doc] if word.lower() in positive])
  neg_words.append([word for word in scrap_docs[doc] if word.lower() in negative])
  positive_score.append(len(pos_words[doc]))
  negative_score.append(len(neg_words[doc]))
  polarity_score.append((positive_score[doc]-negative_score[doc])/((positive_score[doc]+negative_score[doc])+0.000001))
  subjectivity_score.append((positive_score[doc]+negative_score[doc])/((len(scrap_docs[doc]))+0.000001))

In [ ]:
# variables calculations
avg_sent_len = []
percent_complex_words = []
fog_index = []
complex_word_count=[]
syllable_cnt_per_wrd = []
word_count = []
avg_wrd_len = []



def variable_calculation(file):
  with open(os.path.join(scrap_text,file),'r',encoding='ISO-8859-1') as f:
    text = f.read()
    text = re.sub('[^a-zA-Z]','',text)
    sentences = text.split('.')
    no_sent = len(sentences)
    words = [word for word in text.split() if word.lower() not in set(stopwords.words('english'))]
    no_words = len(words)
    length_word = sum(len(word) for word in words)


    complex_words=[]

    for word in words:
       vowels = 'aeiou'
       syllable_count = sum( 1 for letter in word if letter.lower() in vowels)
    if syllable_count > 2:
      complex_words.append(word)
    #complex_word_count = len(complex_words)

    syllable_count = 0
    syllable_words = []
    for word in words:
      if word.endswith('es'):
        word = word[:-2]
      elif word.endswith('ed'):
        word = word[:-2]
      for letter in word:
        if letter.lower() in vowels:
          syllable_count = syllable_count + 1
          if syllable_count >= 1:
            syllable_words.append(word)


    avg_sent_len = no_words/no_sent
    percent_complex_words = len(complex_words)/no_words
    fog_index = 0.4*(avg_sent_len + percent_complex_words)
    complex_word_count =len(complex_words)
    syllable_cnt_per_wrd = syllable_count/len(syllable_words)
    word_count = no_words
    avg_wrd_len = length_word/no_words

    #complex_word_count =len(complex_words)

    return avg_sent_len,percent_complex_words,fog_index,complex_word_count,syllable_cnt_per_wrd,word_count,avg_wrd_len

In [ ]:
for file in os.listdir(scrap_text):
  a,b,c,d,e,f,g= variable_calculation(file)
  avg_sent_len.append(a)
  percent_complex_words.append(b)
  fog_index.append(c)
  complex_word_count.append(d)
  syllable_cnt_per_wrd.append(e)
  word_count.append(f)
  avg_wrd_len.append(g)

In [ ]:
len(avg_sent_len)

114

In [ ]:
# personal pronoun calculation
def pp(file):
  with open(os.path.join(scrap_text,file), 'r') as f:
    text = f.read()
    pronouns = ["I", "we", "my", "ours", "us"]
    s = 0
    for pronoun in pronouns:
      s = s + len(re.findall(r"\b" + pronoun + r"\b", text))
  return s

pp_count = []
for file in os.listdir(scrap_text):
  a = pp(file)
  pp_count.append(a)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Output Data Structure.xlsx to Output Data Structure (2).xlsx


In [ ]:
output_df = pd.read_excel('Output Data Structure (2).xlsx')

In [ ]:
output_df.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
col_values = [positive_score,negative_score,polarity_score,subjectivity_score,avg_sent_len,percent_complex_words,
              fog_index,avg_sent_len,complex_word_count,word_count,syllable_cnt_per_wrd,pp_count,avg_wrd_len]

In [ ]:
for i, col_value in enumerate(col_values):
  output_df.iloc[:,i+2] = col_value

# exporting data to csv file
output_df.to_csv('Output_data.csv')

<ipython-input-114-91e93d5c8a5c>:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  output_df.iloc[:,i+2] = col_value


In [ ]:
output_df.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,84,25,0.541284,0.091906,1.0,1.0,0.8,1.0,1,1,1.0,2,10493.0
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,42,13,0.527273,0.112705,1.0,1.0,0.8,1.0,1,1,1.0,3,4338.0
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,23,27,-0.080000,0.057803,1.0,1.0,0.8,1.0,1,1,1.0,3,7040.0
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,39,27,0.181818,0.070740,1.0,1.0,0.8,1.0,1,1,1.0,6,7760.0
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,39,28,0.164179,0.072354,1.0,1.0,0.8,1.0,1,1,1.0,6,7753.0
